# Challenge 04-B - Retrieval Augmented Generation (RAG) for Unstructured Data


## Introduction

Businesses have a lot of proprietary information that needs to be taken into account when answering user's questions - these cannot always be answered through the data that the GPT models have been trained on. 

In the last notebook, we worked with structured data primarily. A lot of the time, your enterprise data is not limited to just structured formats like CSV files or SQL tables. It may also include unstructured data like PDF documents or images. In fact, your individual documents could have both unstructured and structured data built into them. Extracting information from these diverse formats in a comprehensible manner presents a challenge. Tools like Azure Form Recognizer enable the extraction of data from unstructured sources such as forms or documents. Once the data is extracted into a structured JSON format, then Cognitive Search can be utilized to consolidate the entire information from different data types into indexes, facilitating the retrieval of relevant documents.

In this notebook, we will walk you through a use case of Retrieval Augmented Generation (RAG) that involves working with unstructured data. The RAG approach combines various technologies to enhance the quality and relevance of generated outputs. We will leverage Azure Form Recognizer to process complex documents, utilizing the layout API to extract text and tables effectively. We will utilize Azure Cognitive Search to create an index by configuring semantic search capabilities, enabling the retrieval of relevant document pages. Additionally, embeddings will be incorporated to retrieve content that is more closely aligned with the user's question. Finally, Azure OpenAI's ChatGPT model will utilize the extracted content to generate a more meaningful answer. It is important to emphasize that this grounding process follows the RAG pattern mentioned in the previous notebook and helps eliminate inaccuracies in the generated responses.

Your goals for this challenge are to read through this notebook, run each code block, observe the results, and then be able to answer the questions posed in the student guide.


In [1]:
! pip install "tiktoken==0.5.2" 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# Import Azure Forms Recognizer, Azure Cognitive Search, OpenAI, and other python modules

import os, json, requests, sys, re
import requests
from pprint import pprint
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SemanticSettings
)

from azure.ai.formrecognizer import DocumentAnalysisClient
import openai
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# This is secure and recommended way to load OpenAI resource credentials and deployment names

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['CHAT_MODEL_NAME']
embedding_model=os.environ['EMBEDDING_MODEL_NAME']

**NOTE:** The path in the code cell below is referring to the `/data/unstructured/raw` folder. You may need to update this path if you are running this notebook from a different location then from where you extracted it.

In [4]:
# -- raw data
RAW_DATA_FOLDER= '../data/unstructured/raw'
# -- extracted json file 
EXTRACTED_DATA_FOLDER = '../data/unstructured/extracted'

In [5]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

We want to extract the data from our unstructured data into a more readable format for the model to understand. The Form Recognizer tool helps us do so by leveraging the prebuilt layout models. Here, we primarily are working with PDFs but we could also have JPG and PNG formats that the form recognizer tool also supports.

For each document, we want to specify the way information is being extracted. For example in this use case, each document has many pages. To keep track of the pages, we store them in page_number. We also want to extract the content for each page and drop it in a page_context field.

In [6]:
def extract_local_single_file(file_name: str):
    not_completed = True
    while not_completed:
        with open(file_name, "rb") as f:
            poller = document_analysis_client.begin_analyze_document(
                "prebuilt-layout", document=f
            )
            not_completed=False
    result = poller.result()
    return get_page_content(file_name, result)

def extract_files( folder_name: str, destination_folder_name: str):
    os.makedirs(destination_folder_name, exist_ok=True)
    for file in os.listdir(folder_name):
        if file[-3:].upper() in ['PDF','JPG','PNG']:
            print('Processing file:', file, end='')
        
            page_content = extract_local_single_file(os.path.join(folder_name, file))
            output_file = os.path.join(destination_folder_name, file[:-3] +'json')
            print(f'  write output to {output_file}')
            with open(output_file, "w") as f:
                f.write(json.dumps(page_content))


def get_page_content(file_name:str, result):
    page_content = []
    for page in result.pages:
        all_lines_content = []
        for line_idx, line in enumerate(page.lines):
            all_lines_content.append(' '.join([word.content for word in line.get_words()]))
        page_content.append({'page_number':page.page_number, 
                                'page_content':' '.join(all_lines_content)})
    return {'filename':file_name, 'content':page_content}





In [7]:
extract_files(RAW_DATA_FOLDER, EXTRACTED_DATA_FOLDER)

Processing file: Precise_Zero-Shot_Dense_Retrieval_without_Relevance_Labels.pdf  write output to ../data/unstructured/extracted/Precise_Zero-Shot_Dense_Retrieval_without_Relevance_Labels.json
Processing file: Prefix-Tuning_Optimizing_Continuous_Prompts_for_Generation.pdf  write output to ../data/unstructured/extracted/Prefix-Tuning_Optimizing_Continuous_Prompts_for_Generation.json
Processing file: Power_of_Scale_for_Parameter-Efficient_Prompt_Tuning.pdf  write output to ../data/unstructured/extracted/Power_of_Scale_for_Parameter-Efficient_Prompt_Tuning.json


## More About our data

For this walkthrough, we will take a look at various Research Papers on LLM topics in PDF documents. This includes topics like autoprompting, chain of thought prompting, precise zero shot dense retrival, and more. This dataset contains various unstructured formats such as text, tables, graphs, and formulas.

## Data Description

The relevant schema for our work today consists of 

- document_id
- document_name
- file_path
- page_number
- page_text


In [8]:
documents=[]
for file in os.listdir(EXTRACTED_DATA_FOLDER):
    with open(os.path.join(EXTRACTED_DATA_FOLDER, file)) as f:
        page_content= json.loads(f.read())
    documents.extend(
        [
            {
                'document_id':page_content['filename'].split('\\')[-1].split('.')[0] + '-' + str(page['page_number']),
                'document_name':page_content['filename'].split('\\')[-1],
                'file_path':page_content['filename'],              
                'page_number':page['page_number'],
                'page_text':page['page_content']
            }
            for page in page_content['content']
        ]
    )

In [9]:
#Example of a single page of research paper file that will be indexed in Azure Cognitive Search
documents[1]

{'document_id': '-2',
 'document_name': '../data/unstructured/raw/Power_of_Scale_for_Parameter-Efficient_Prompt_Tuning.pdf',
 'file_path': '../data/unstructured/raw/Power_of_Scale_for_Parameter-Efficient_Prompt_Tuning.pdf',
 'page_number': 2,
 'page_text': 'Model Tuning Pre-trained Model (11B params) Prompt Tuning a1 a2 Task A Batch b1 Task B Batch c1 Task C c2 Batch Mixed-task Batch Task A Model (11B params) A a1 A C c1 b1 a2 B A B Task B Model (11B params) -- C c2 C Task Prompts (20K params each) Task C Model (11B params) ---- Pre-trained Model (11B params) Figure 2: Model tuning requires making a task- specific copy of the entire pre-trained model for each downstream task and inference must be performed in separate batches. Prompt tuning only requires stor- ing a small task-specific prompt for each task, and enables mixed-task inference using the original pre- trained model. With a T5 “XXL” model, each copy of the tuned model requires 11 billion parameters. By contrast, our tuned pr

This section will focus on Cognitive Search and the following topics:
1. Creating an index client
2. Defining the index fields with necessary attributes
3. Creating a semantic configuration
4. Loading our index with the document pages

In [10]:
# Create an SDK client
service_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")   
key = os.getenv("AZURE_COGNITIVE_SEARCH_KEY")
credential = AzureKeyCredential(key)

index_name = os.getenv("AZURE_COGNITIVE_SEARCH_DOC_INDEX_NAME")

index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
index_client

In [12]:
fields = [
    SimpleField(name="document_id", type=SearchFieldDataType.String, key=True),
    SimpleField(name="page_number", type=SearchFieldDataType.Int64),
    SimpleField(name="file_path", type=SearchFieldDataType.String),
    SearchableField(name="document_name", type=SearchFieldDataType.String,
                searchable=True, retrievable=True),
    SearchableField(name="page_text", type=SearchFieldDataType.String,
                filterable=True, searchable=True, retrievable=True),
]

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="document_id"),
        prioritized_keywords_fields=[SemanticField(field_name="document_name")],
        prioritized_content_fields=[SemanticField(field_name="page_text")]
    )
)


# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 research-paper-index-labp created


In [13]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 144 documents


In [14]:
len(result)

144

Here we see Azure Cognitive Search in action! We can retrive the most relevant documents out of all the ones that we are working with. 

In [15]:
query = "What is prompt tuning?"
count = 10
results = search_client.search(search_text=query, top=count, include_total_count=True)
page_chunks = []
citations = []
for result in results:
    page_chunks.append(result['page_text'])
    citations.append(result['document_name'])
    
    

In [16]:
embed_df = pd.DataFrame(page_chunks, columns = ["page_chunks"]) #datframe with document chunks
embed_df

,page_chunks
0,Table 24: Few-shot exemplars for full chain of...
1,Self-Consistency Improves Chain of Thought Rea...
2,Self-Consistency Improves Chain of Thought Rea...
3,Table 25: Few-shot exemplars for full chain of...
4,F Appendix: Input/Output Examples Table 13: Ex...
5,Self-Consistency Improves Chain of Thought Rea...
6,Self-Consistency Improves Chain of Thought Rea...
7,Table 23: Few-shot exemplars for full chain of...
8,likely that models could arrive at the correct...
9,Self-Consistency Improves Chain of Thought Rea...


Once we have the most relevant documents, let us create embeddings for all the page chunks. This will help us find the most similar documents to our given user query.

In [17]:
# Handling Rate Limits

from openai.error import RateLimitError
from time import sleep


def get_embedding(text: str, engine: str = "text-embedding-ada-002"):
    count=0
    while True:
        try:
            embedding = openai.Embedding().create(input=[text], engine=engine)["data"][0]["embedding"]
            break;
        except RateLimitError:
            count+=1
            #print(f'RateLimitError Count: {count}')
            sleep(2)            
    return np.array(embedding).astype(np.float32)

def get_completion(prompt, model="gpt-35-turbo"): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [18]:
#Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
embed_df['embedding'] = embed_df["page_chunks"].apply(lambda page_text : get_embedding(page_text, engine = embedding_model))

In [19]:
embed_df

,page_chunks,embedding
0,Table 24: Few-shot exemplars for full chain of...,"[0.013712603, 0.012631297, 0.030331643, -0.004..."
1,Self-Consistency Improves Chain of Thought Rea...,"[-0.009956691, 0.017103354, 0.0029062866, -0.0..."
2,Self-Consistency Improves Chain of Thought Rea...,"[0.012837319, 0.023359483, 0.007215793, -0.032..."
3,Table 25: Few-shot exemplars for full chain of...,"[-0.0047118575, 0.011542224, -0.004891167, -0...."
4,F Appendix: Input/Output Examples Table 13: Ex...,"[-0.0065279542, 0.029688586, 0.009669996, -0.0..."
5,Self-Consistency Improves Chain of Thought Rea...,"[0.0039505716, 0.014943616, 0.009306868, -0.02..."
6,Self-Consistency Improves Chain of Thought Rea...,"[0.0022468215, 0.018690253, 0.009998872, -0.02..."
7,Table 23: Few-shot exemplars for full chain of...,"[-0.0074769733, 0.0077185873, 0.014000551, -0...."
8,likely that models could arrive at the correct...,"[0.028560385, 0.006772641, 0.021386577, -0.030..."
9,Self-Consistency Improves Chain of Thought Rea...,"[0.00010377741, 0.015690768, 0.0045664157, -0...."


In [20]:
query_embedding = get_embedding(query, engine=embedding_model)
embed_df["similarities"] = embed_df['embedding'].apply(lambda page_embedding: cosine_similarity(page_embedding, query_embedding))

top_results = (
    embed_df.sort_values("similarities", ascending=False)
    .reset_index(drop=True)
    .head(3)
)
top_results

,page_chunks,embedding,similarities
0,Self-Consistency Improves Chain of Thought Rea...,"[0.0022468215, 0.018690253, 0.009998872, -0.02...",0.756298
1,Self-Consistency Improves Chain of Thought Rea...,"[-0.009956691, 0.017103354, 0.0029062866, -0.0...",0.747208
2,Table 24: Few-shot exemplars for full chain of...,"[0.013712603, 0.012631297, 0.030331643, -0.004...",0.744863


In [21]:
prompt = f"""
Provided below are user query and list of extracted pages from research papers separated by triple backticks.
Your task is to extract key pieces of information from that list based on the user query and phrase that as a comprehensive answer. 

User Query: ```{query}```
List of Extracted Pages: ```{top_results['page_chunks'].to_list()}```

Answer:
"""

print(prompt)


Provided below are user query and list of extracted pages from research papers separated by triple backticks.
Your task is to extract key pieces of information from that list based on the user query and phrase that as a comprehensive answer. 

User Query: ```What is prompt tuning?```
List of Extracted Pages: ```['Self-Consistency Improves Chain of Thought Reasoning in Language Models GSM8K accuracy Different sets of prompts (x40) 58.9 Prompt-permutation (x40) 59.6 Self-consistency (x40) 74.4 Self-consistency + different sets of prompts (x40) 75.4 Self-consistency + prompt-permutation (x40) 73.8 Table 11: Combining self-consistency with other ensembling strategies. permutations to compare with self-consistency with 40 paths, all experiments are based on PaLM- 540B). Question Greedy Decode Sampled Path 1 Sampled Path 2 [MultiArith] Sam had 10 socks. If he threw away 3 old ones that didn’t fit and bought 36 new ones, how many socks would he have? Sam had 10 socks. He threw away 3 old one

In [22]:
response = get_completion(prompt, chat_model)
print(response)

Prompt tuning is a technique used to improve the performance of language models in Type 2 tasks such as arithmetic, logical, and commonsense reasoning. It involves generating diverse reasoning paths and using self-consistency to improve the accuracy of the final answer. Self-consistency works under imperfect prompts, equation prompts, and zero-shot chain-of-thought for GSM8K. The consistency is correlated with the model's accuracy. The approach is far simpler and requires no additional training.


In [23]:

def query_search(query, count=10):
    results = search_client.search(search_text=query, top=count, include_total_count=True)
    page_chunks = []
    for result in results:
        page_chunks.append(result['page_text'])
        
    #Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
    embed_df['embedding'] = embed_df["page_chunks"].apply(lambda page_text : get_embedding(page_text, engine = embedding_model))

    query_embedding = get_embedding(query, engine=embedding_model)
    embed_df["similarities"] = embed_df['embedding'].apply(lambda page_embedding: cosine_similarity(page_embedding, query_embedding))

    top_results = (
        embed_df.sort_values("similarities", ascending=False)
        .reset_index(drop=True)
        .head(3)
    )
    
    prompt = f"""
    Provided below are user query and list of extracted pages from research papers separated by triple backticks.
    Your task is to extract key pieces of information from that list based on the user query and phrase that as a comprehensive answer. 

    User Query: ```{query}```
    List of Extracted Pages: ```{top_results['page_chunks'].to_list()}```

    Answer:
    """
    
    response = get_completion(prompt, chat_model)
    return response

In [24]:
answer = query_search("How does prompt tuning work?", 5)
print(answer)

Prompt tuning works by using the self-consistency method, which involves prompting a language model with a set of manually written chain-of-thought exemplars, replacing the "greedy decode" in chain-of-thought prompting by sampling from the language model's decoder to generate a diverse set of reasoning paths, and marginalizing out the reasoning paths and aggregating by choosing the most consistent answer in the final answer set. This method improves the performance of language models on a wide range of arithmetic and commonsense reasoning tasks, including GSM8K, SVAMP, AQuA, StrategyQA, and ARC-challenge, and is robust to sampling strategies and imperfect prompts.


In [25]:
answer = query_search("what is a fully zero-shot dense retrieval system?", 10)
print(answer)

A fully zero-shot dense retrieval system is a system that can accurately retrieve information without any training or fine-tuning on a specific task. Self-consistency is a method that improves chain-of-thought reasoning in language models by introducing diversity in the reasoning process through sampling. It has been shown to work for zero-shot chain-of-thought and improves results significantly. The approach can be applied to problems where the final answer is from a fixed answer set. The method has been tested on a range of reasoning benchmarks and has been found to robustly improve reasoning accuracy for every language model considered.
